# Package Loading

In [1]:
import pandas as pd
import numpy as np

from typing import List

# Data Lodaing

In [2]:
# Data Loading
data = pd.read_excel('../raw_data/농협양곡 매출데이터(2015년).xlsx')

# 정정취소구분가 "정상"인 데이터만 추출
data = data[data['정정취소구분']=='정상']
data.head()

"\n#정상만 인덱싱\ndata = data[data['정정취소구분']=='정상']\ndata\n"

In [3]:
data['정정취소구분'].describe()

count     223538
unique         1
top           정상
freq      223538
Name: 정정취소구분, dtype: object

In [4]:
data['양곡분류구분'].describe()

count     223538
unique         4
top           잡곡
freq      125790
Name: 양곡분류구분, dtype: object

## 품목명 분류 전처리

In [5]:
# 백미를 담을 리스트
tmp_rice: List[str] = []

# 잡곡을 담을 리스트
tmp_grain: List[str] = [] 

#### 미곡

In [6]:
rice = data[data['양곡분류구분']=='미곡']
rice.reset_index(drop=True, inplace=True)

tmp_rc = list(set(rice['품목명']))

# 찹쌀 및 잡곡 품목 확인 
for num in range(len(tmp_rc)):
    name = tmp_rc[num]    
    
    if ('현미' in name) or ('흑미' in name) or ('잡곡' in name) or ('초록통쌀' in name):
        tmp_grain.append(name)
        
    else :
        tmp_rice.append(name)

#### 선물세트

In [7]:
gift = data[data['양곡분류구분']=='선물세트'].reset_index()

tmp_gift = list(set(gift['품목명']))

for num in range(len(tmp_gift)):
    
    name = tmp_gift[num]
    
    tmp_grain.append(name)

#### PB상품

In [8]:
# pb상품 중 쌀 포함된 품목 찾기
pb = data[data['양곡분류구분']=='PB상품'].reset_index()

tmp_pb = list(set(pb['품목명']))

# 백미 =>'갯반닥서해일미 20kg','농협쌀(진공)500g' / 찹쌀제외 / 나머지 잡곡으로 확인

for num in range(len(tmp_pb)):
    
    name = tmp_pb[num]
    
    if ('현미' in name) or ('흑미' in name) or ('잡곡' in name):
        tmp_grain.append(name)
    
    elif ('찹쌀' in name) or (name  == '갯반닥서해일미 20kg'):
        tmp_rice.append(name)

    else:
        tmp_grain.append(name)


#### 잡곡

In [9]:
grain = data[data['양곡분류구분']=='잡곡']

tmp_gr = list(set(grain['품목명']))

for num in range(len(tmp_gr)):
    
    name = tmp_gr[num]

    if ('햅쌀' in name) or ('올벼쌀' in name) or ('상록수쌀' in name) or \
            ('당진해나루쌀' in name) or ('대숲깨끗한쌀' in name) or ('선산쌀' in name) or ('오대쌀' in name):
        tmp_rice.append(name)
        
    elif ('현미' in name) or ('흑미' in name) or ('잡곡' in name) or ('찹쌀보리' in name) or ('검정' in name) or ('찹쌀세트' == name):
        tmp_grain.append(name)
        
    elif '청미2kg/봉지' in name:
        continue
        
    elif '벼' in name or '찹쌀' in name:
        tmp_rice.append(name) 
    
    else:
        tmp_grain.append(name)


In [10]:
tmp_rice = list(set(tmp_rice))
len(tmp_rice)

1680

In [11]:
tmp_grain = list(set(tmp_grain))
len(tmp_grain)

2549

In [12]:
data_copy = data.copy()
data_copy.loc[data_copy['품목명'].isin(tmp_rice), '품종'] = '미곡'
data_copy.loc[data_copy['품목명'].isin(tmp_grain), '품종'] = '잡곡'
data_copy.head()

,사업장,매출일,매출번호,정정취소구분,거래처유형,매출처명,품목명,생산년도,규격명,양곡분류구분,매출수량,환산중량,매출단가,공급금액,품종
0,본사,2015-03-02,8,정상,계통가공처(매입),충서원예농협 공판장지점(판매),하나가득흑미1kg(20),2014,1kg,PB상품,100.0,100.0,3700.0,370000,잡곡
1,본사,2015-03-02,8,정상,계통가공처(매입),충서원예농협 공판장지점(판매),하나가득찰보리쌀1kg(20),2014,1kg,PB상품,100.0,100.0,2300.0,230000,잡곡
2,본사,2015-03-02,10,정상,계통마트(농축협),한국양봉농협 신풍역판매장,동송신선오대쌀(인증) 20kg,2014,20kg,미곡,3.0,60.0,60000.0,180000,미곡
3,본사,2015-03-02,10,정상,계통마트(농축협),한국양봉농협 신풍역판매장,동송신선오대쌀(인증) 10kg,2014,10kg,미곡,3.0,30.0,31000.0,93000,미곡
4,본사,2015-03-02,10,정상,계통마트(농축협),한국양봉농협 신풍역판매장,신평맛좋은당진쌀20kg,2014,20kg,미곡,5.0,100.0,43000.0,215000,미곡


In [13]:
data_copy[data_copy['품종'].isnull()]

,사업장,매출일,매출번호,정정취소구분,거래처유형,매출처명,품목명,생산년도,규격명,양곡분류구분,매출수량,환산중량,매출단가,공급금액,품종
15613,본사,2015-03-25,40,정상,계통마트(도매),(주)농협유통전주유통센터(도매),청미2kg/봉지,2014,2Kg,잡곡,30.0,60.0,8944.0,268320,NaN
41443,본사,2015-04-30,143,정상,계통마트,(주)농협유통양재점,청미2kg/봉지,2014,2Kg,잡곡,10.0,20.0,8944.0,89440,NaN
60431,본사,2015-05-28,232,정상,계통마트,(주)농협유통양재점,청미2kg/봉지,2014,2Kg,잡곡,10.0,20.0,8944.0,89440,NaN
80136,본사,2015-06-24,193,정상,계통마트,(주)농협유통양재점,청미2kg/봉지,2014,2Kg,잡곡,10.0,20.0,8944.0,89440,NaN
105019,본사,2015-07-27,47,정상,계통마트(도매),(주)농협유통전주유통센터(도매),청미2kg/봉지,2014,2Kg,잡곡,20.0,40.0,8944.0,178880,NaN
108764,본사,2015-07-30,107,정상,계통마트,(주)농협유통양재점,청미2kg/봉지,2014,2Kg,잡곡,10.0,20.0,8944.0,89440,NaN
134077,본사,2015-09-02,203,정상,계통마트,(주)농협유통양재점,청미2kg/봉지,2014,2Kg,잡곡,10.0,20.0,8944.0,89440,NaN
134989,본사,2015-09-03,48,정상,계통마트(도매),(주)농협유통전주유통센터(도매),청미2kg/봉지,2014,2Kg,잡곡,20.0,40.0,8944.0,178880,NaN
151210,본사,2015-09-23,124,정상,계통마트(도매),(주)농협유통전주유통센터(도매),청미2kg/봉지,2014,2Kg,잡곡,20.0,40.0,8944.0,178880,NaN
162623,본사,2015-10-10,33,정상,계통마트,(주)농협유통양재점,청미2kg/봉지,2015,2Kg,잡곡,10.0,20.0,8944.0,89440,NaN


In [14]:
data_copy.dropna(inplace=True)
data_copy.reset_index(drop=True, inplace=True)
data_copy

,사업장,매출일,매출번호,정정취소구분,거래처유형,매출처명,품목명,생산년도,규격명,양곡분류구분,매출수량,환산중량,매출단가,공급금액,품종
0,본사,2015-03-02,8,정상,계통가공처(매입),충서원예농협 공판장지점(판매),하나가득흑미1kg(20),2014,1kg,PB상품,100.0,100.0,3700.0,370000,잡곡
1,본사,2015-03-02,8,정상,계통가공처(매입),충서원예농협 공판장지점(판매),하나가득찰보리쌀1kg(20),2014,1kg,PB상품,100.0,100.0,2300.0,230000,잡곡
2,본사,2015-03-02,10,정상,계통마트(농축협),한국양봉농협 신풍역판매장,동송신선오대쌀(인증) 20kg,2014,20kg,미곡,3.0,60.0,60000.0,180000,미곡
3,본사,2015-03-02,10,정상,계통마트(농축협),한국양봉농협 신풍역판매장,동송신선오대쌀(인증) 10kg,2014,10kg,미곡,3.0,30.0,31000.0,93000,미곡
4,본사,2015-03-02,10,정상,계통마트(농축협),한국양봉농협 신풍역판매장,신평맛좋은당진쌀20kg,2014,20kg,미곡,5.0,100.0,43000.0,215000,미곡
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223533,본사,2015-12-31,356,정상,기타,하루두부,콩(백태)/1Kg/PP대,2014,1Kg,잡곡,520.0,520.0,4000.0,2080000,잡곡
223534,본사,2015-12-31,356,정상,기타,하루두부,하나가득혼합잡곡PLUS14곡2kg(10),2014,2kg,PB상품,1.0,2.0,10890.0,10890,잡곡
223535,본사,2015-12-31,357,정상,기타,양곡부-경제통합매출용(비사업),하나가득찰보리쌀4kg(5),2015,4kg,PB상품,-25.0,-100.0,7600.0,-190000,잡곡
223536,본사,2015-12-31,358,정상,계통마트(식자재),(주)농협하나로유통 성남점(식자재),부발임금(표)쌀4/국산,2015,4kg,미곡,1.0,4.0,13635.0,13635,미곡


# CSV 저장

In [15]:
data_copy.to_csv('./preprocessed_data/preprocessed_data_2015.csv', index=False)